<a href="https://colab.research.google.com/github/ThomasAlbin/Astroniz-YT-Tutorials/blob/main/CompressedCosmos/CompressedCosmos_Geminids_and_Phaethon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The Geminids and Phaethon

In this short *Compressed Cosmos* tutorial we will take a look at the so-called Dsh parameter of meteor science.

The Dsh value uses "some" orbital elements and computes a dynamical relationship between objects. Like e.g. the Geminids and (3200) Phaethon.

What will we find?

In [10]:
import math

import pandas as pd
meteor_df = pd.read_parquet("edmond_meteor.parquet")

In [11]:
meteor_df

,q,e,i,nod,pi
7600,0.431021,0.824703,1.935817,60.523758,104.295120
7601,0.347479,0.527846,83.641457,240.555817,331.171814
7602,0.957224,0.939699,75.295074,240.582077,200.518814
7603,0.932860,0.597927,11.431081,60.592392,31.577040
7604,0.105448,0.981881,27.400726,60.607029,143.411194
...,...,...,...,...,...
254065,0.153609,0.917156,25.257166,279.971863,319.800354
254066,0.928605,0.655110,13.873104,279.980469,210.758469
254067,0.973011,0.573889,71.939285,279.983063,193.797287
254068,0.251325,0.965385,11.449976,99.988495,121.033211


The $D_{SH}$ parameter, found in "Meteoroid streams that trace to candidate dormant comets" by Peter Jenniskens

$D^2 = (e_1 - e_2)^2 + (q_1 - q_2)^2 + \left[2 \sin\left(\frac{i_1 - i_2}{2}\right)\right]^2$

$D_{SH}^2 = D^2 + \sin(i_1) \sin(i_2)\left[2 \times \sin\left(\frac{\Omega_1 - \Omega_2}{2}\right)\right]^2 + \left[\frac{e_1 + e_2}{2}\right]^2 \times \left[2 \times \sin\left(\frac{\Pi_{12}}{2}\right)\right]^2$

$\Pi_{12} = (\omega_1 - \omega_2) + 2 \arcsin\left[\cos\left(\frac{i_1 - i_2}{2}\right) \times \sin\left(\frac{\Omega_1 - \Omega_2}{2}\right) \times \sec\left(\frac{I_{12}}{2}\right)\right]$

(where the sign of $\arcsin$ should be opposite when $|\Omega_1 - \Omega_2| > 180^\circ$. The angle between the orbital planes is:)

$I_{12} = \arccos[\cos(i_1) \cos(i_2) + \sin(i_1) \sin(i_2) \cos(\Omega_1 - \Omega_2)]$

In [12]:
def sec(x):
    # Added protection against division by zero (though rare in this context)
    c = math.cos(x)
    if abs(c) < 1e-15:
        return 1e15 # Return a large number instead of crashing
    return 1 / c

def clamp(value, min_val=-1.0, max_val=1.0):
    """Restricts a value to a given range."""
    return max(min_val, min(value, max_val))

def calculate_d_criterion(orbit1, orbit2):
    """
    Calculates the Southworth & Hawkins D-Criterion (D_SH) for two orbits.
    Includes numerical stability fixes for math domain errors.
    """

    # 1. Unpack and convert angles to radians
    q1, e1 = orbit1['q'], orbit1['e']
    i1 = math.radians(orbit1['i'])
    node1 = math.radians(orbit1['nod'])
    peri1 = math.radians(orbit1['pi'])

    q2, e2 = orbit2['q'], orbit2['e']
    i2 = math.radians(orbit2['i'])
    node2 = math.radians(orbit2['nod'])
    peri2 = math.radians(orbit2['pi'])

    # 2. Calculate initial difference terms
    term1 = (e2 - e1)**2
    term2 = (q2 - q1)**2
    term3 = (2 * math.sin((i1 - i2)/2))**2

    D_squared = term1 + term2 + term3

    # 3. Calculate Mutual Inclination (I21)
    cos_I21_val = (math.cos(i1) * math.cos(i2) +
                   math.sin(i1) * math.sin(i2) * math.cos(node2 - node1))

    # FIX 1: Use clamp helper
    I21 = math.acos(clamp(cos_I21_val))

    # 4. Calculate Difference in Longitude of Perihelion (Omega21 / Pi_21)
    # The argument for asin can slightly exceed 1.0 due to float precision
    asin_arg = (math.cos((i1 - i2)/2) * math.sin((node2 - node1)/2) * sec(I21/2))

    # FIX 2: Clamp the asin argument
    Omega21 = (peri2 - peri1) + 2 * math.asin(clamp(asin_arg))

    # 5. Final Calculation
    # Note: Added 'abs' to sqrt input just in case of negative float drift (very rare here)
    dsh_squared = (D_squared +
                   math.sin(i1) * math.sin(i2) * (2 * math.sin((node1 - node2)/2))**2 +
                   ((e1 + e2)/2)**2 * (2 * math.sin(Omega21/2))**2)

    return math.sqrt(abs(dsh_squared))

In [13]:
# Orbital elements of (3200) Phaethon (Wikipedia)
phaethon_orbit = {
                    'q': 0.13998,   # AU
                    'e': 0.88990,
                    'i': 22.260,    # degrees
                    'nod': 265.22, # degrees
                    'pi': 322.19  # degrees
                 }

In [14]:
# Compute the DsH value for each dataframe entry:
meteor_df['dsh'] = meteor_df.apply(lambda row: calculate_d_criterion(phaethon_orbit,
                                                                     row), axis=1)

In [15]:
# How many meteors are present in the "almost december dataset?"
len(meteor_df)

40812

In [16]:
# "Linked" with Phaethon
meteor_df.loc[meteor_df["dsh"]<0.2]

,q,e,i,nod,pi,dsh
7721,0.140188,0.880434,20.865963,256.663025,326.287079,0.091855
7724,0.154110,0.877534,19.799000,256.854218,323.590881,0.128309
7727,0.117439,0.907503,21.963938,256.896912,328.354309,0.070562
7730,0.132895,0.896303,22.112980,256.920044,326.123016,0.087879
7738,0.154852,0.882223,20.884926,256.987823,322.946991,0.130361
...,...,...,...,...,...,...
253664,0.114484,1.013349,22.619556,266.757904,319.082458,0.129281
253860,0.228018,0.920910,19.953539,276.085480,307.031189,0.139839
253917,0.241995,0.887533,15.446727,277.113037,307.112213,0.176912
253946,0.164875,0.997548,29.243382,277.974762,311.814819,0.194249
